# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-11-10T00:00:00'
tend = '2017-11-11T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')

#tstart.strftime('%Y:%j:%H:%M:%S')


---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [8]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_314to315.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits


,visible,occulted
0,2017-11-10 01:29:53,2017-11-10 02:25:55
1,2017-11-10 03:06:32,2017-11-10 04:02:34
2,2017-11-10 04:43:11,2017-11-10 05:39:14
3,2017-11-10 06:19:50,2017-11-10 07:15:53
4,2017-11-10 07:56:30,2017-11-10 08:52:32
5,2017-11-10 09:33:09,2017-11-10 10:29:11
6,2017-11-10 11:09:48,2017-11-10 12:05:51
7,2017-11-10 12:46:27,2017-11-10 13:42:30
8,2017-11-10 14:23:06,2017-11-10 15:19:09
9,2017-11-10 15:59:45,2017-11-10 16:55:48


In [5]:
#from nustar_planning.lunar_planning import position, position_shift
import nustar_planning.lunar_planning as lunar_planning
from importlib import reload
reload(lunar_planning)

<module 'nustar_planning.lunar_planning' from '/Users/bwgref/anaconda/lib/python3.5/site-packages/nustar_planning-0.1.dev65-py3.5.egg/nustar_planning/lunar_planning.py'>

In [6]:
reload(lunar_planning)
sub = orbits.iloc[11:12].reset_index()
sub.head()
#lunar_planning.position(sub, steps = 5, parallax_correction=False)

,index,visible,occulted
0,11,2017-11-10 19:13:03,2017-11-10 20:09:07


In [9]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
reload(lunar_planning)
lunar_planning.position_shift(sub, parallax_correction=True, pa = 172.0*u.deg, dt = 3*u.s,
                              outfile='lunar_20171110_v2.txt', show=True)

2017:314:19:08:03 RA: 140.90112  Dec: 15.43836

2017:314:19:10:15 RA: 141.00035  Dec: 15.46896

2017:314:19:12:57 RA: 141.08768  Dec: 15.52305

2017:314:19:16:51 RA: 141.08397  Dec: 15.62348

2017:314:19:23:00 RA: 141.00202  Dec: 15.68555

2017:314:19:26:39 RA: 140.90697  Dec: 15.72968

2017:314:19:29:21 RA: 140.80822  Dec: 15.76397

2017:314:19:31:36 RA: 140.70771  Dec: 15.79175

2017:314:19:33:36 RA: 140.60595  Dec: 15.81467

2017:314:19:35:27 RA: 140.50354  Dec: 15.83349

2017:314:19:37:12 RA: 140.39990  Dec: 15.84880

2017:314:19:38:54 RA: 140.29371  Dec: 15.86092

2017:314:19:40:36 RA: 140.18969  Dec: 15.86944

2017:314:19:42:15 RA: 140.08310  Dec: 15.87473

2017:314:19:43:57 RA: 139.97866  Dec: 15.87636

2017:314:19:45:39 RA: 139.87215  Dec: 15.87404

2017:314:19:47:27 RA: 139.76618  Dec: 15.86708

2017:314:19:49:21 RA: 139.66152  Dec: 15.85444

2017:314:19:51:24 RA: 139.55875  Dec: 15.83417

2017:314:19:53:42 RA: 139.46000  Dec: 15.80207

2017:314:19:56:27 RA: 139.37340  Dec: 15